### Import all the libraries

In [12]:
# Importing the necessary libraries for selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
# Import SQLAlchemy stuff
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
# Import the rest of the libraries
import csv, os, re, time

### Necessary functions

In [13]:
# Function to find the first word before any special character in a string
def find_string_before_special(input_string):
    """Function to find the first word before any special character in a string

    Args:
        input_string (string): the features that the current car has on this page

    Returns:
        string: returns the first word before any special character
    """
    special_characters = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    for i, char in enumerate(input_string):
        if char in special_characters:
            return input_string[:i].strip()
    return input_string

# Function to check if a car has certain features
def check_car_features(input_list, feature_list):
    """a function to check if a car has certain features and will add a "has_" as a prefix to the input

    Args:
        input_list (list): the features that the current car has on this page
        feature_list (list): list of all the features

    Returns:
        dictionary: return a dictionary with the key as the feature name and the value True or False depending on whether the car has certain features
    """
    feature_dict = {}
    for feature in feature_list:
        key_name = "Has_" + feature.replace(" ", "_")
        feature_dict[key_name] = feature in input_list
    return feature_dict


### Base class for creating the database for sqlalchemy

In [14]:
# Create a model for Car
Base = declarative_base()

class Car(Base):
    __tablename__ = 'cars_db'
    
    Id = Column(Integer, primary_key=True)
    Make = Column(String, nullable=True)
    Model = Column(String, nullable=True)
    Trim = Column(String, nullable=True)
    Variant = Column(String, nullable=True)
    Body_type = Column(String, nullable=True)
    Transmission = Column(String, nullable=True)
    Drivetrain = Column(String, nullable=True)
    Fuel_type = Column(String, nullable=True)
    Year_of_manufacture = Column(Integer, nullable=True)
    Mileage_in_KM = Column(Integer, nullable=True)
    Mileage_analysis = Column(String, nullable=True)
    Condition = Column(String, nullable=True)
    Number_of_seats = Column(Integer, nullable=True)
    Number_of_doors = Column(Integer, nullable=True)
    Colour = Column(String, nullable=True)
    Province = Column(String, nullable=True)
    Price_analysis = Column(String, nullable=True)
    Has_air_conditioning = Column(Boolean)
    Has_cruise_control = Column(Boolean)
    Has_parking_assistant = Column(Boolean)
    Has_navigation_system = Column(Boolean)
    Has_bluetooth = Column(Boolean)
    Has_alloy_wheels = Column(Boolean)
    Has_trailer_hitch = Column(Boolean)
    Has_push_button_start = Column(Boolean)
    Has_sunroof = Column(Boolean)
    Price = Column(Integer, nullable=True)


### Connect to the SQLalchemy database

In [15]:
current_directory = os.getcwd()
db_path = os.path.join(current_directory, '..', 'database', 'car_data.db')
engine = create_engine(f'sqlite:///{db_path}')
Base.metadata.create_all(engine)

### Creating a new database session

In [16]:
Session = sessionmaker(bind=engine)
session = Session()

### Opening Kijiji auto and click on the Submit button on the home page

In [17]:
# Initialize WebDriver and navigate to the website
driver = webdriver.Chrome()
driver.get("https://www.kijijiautos.ca/")
driver.implicitly_wait(2)

# Click the first result button
buttons = driver.find_elements(By.CSS_SELECTOR, "button[data-testid='quickSearchResultButton'][type='submit']")
buttons[0].click()


### Locate the main div that holds all the cars articles

In [18]:
# Locate the main div containing the articles
main_div = driver.find_element(By.XPATH, '//*[@data-testid="ListItemPage-0"]')

# Initialize article index
article_index = 0

### Collect the car data on the page

In [19]:
# Loop to go through each article
while True:
    articles = main_div.find_elements(By.XPATH, '//*[@data-testid="SearchResultListItem"]')

    if not articles or article_index >= len(articles):
        break
    articles[article_index].click()

    # Do some tasks on the car page (this part is up to you)
    
    try:
        # Find the element using CSS Selector
        address_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='SellerCardTwoColumn'] span")
    except NoSuchElementException:
        print("Address elements not found")
        
    try:
        # Get the text from the second span element
        address = address_elements[1].text
        # Regex pattern to extract the province code
        province_pattern = r',\s([A-Z]{2})\s'
        province = re.search(province_pattern, address).group(1)
        province = province.lower()
    except IndexError:
        province = None
    province_abbreviation = {
        'nl': 'newfoundland and labrador',
        'pe': 'prince edward island',
        'ns': 'nova scotia',
        'nb': 'new brunswick',
        'qc': 'quebec',
        'on': 'ontario',
        'mb': 'manitoba',
        'sk': 'saskatchewan',
        'ab': 'alberta',
        'bc': 'british columbia',
        'yt': 'yukon',
        'nt': 'northwest territories',
        'nu': 'nunavut'
        }
    province = province_abbreviation.get(province, "Unknown Province")

    ### Price ###
    price = None
    try:
        # Find the element using CSS Selector
        price_element = driver.find_element(By.CSS_SELECTOR, "span[data-testid='listing-basic-info-section-price'] span")
        price = price_element.text
    except NoSuchElementException:
        print("Price element not found")
    if price != None:
        price = int(price_element.text[1:].replace(',', ''))
        
    ### Price rating ###
    try:
        price_analysis_element = driver.find_element(By.CSS_SELECTOR, "button[data-testid='priceRatingButton'] span")  
        price_analysis = price_analysis_element.text.lower()
    except NoSuchElementException:
        price_analysis = None
        
    ### Quick fact selector ###
    try:
        quick_fact_elements = driver.find_elements(By.CSS_SELECTOR, "ul[data-testid='quickFact'] span")
    except NoSuchElementException:
        print("Quick fact elements not found")
        
    ### car_condition ###
    try:
        condition = quick_fact_elements[0].text.lower()
        if condition == "-":
            condition = None
    except IndexError:
        condition = None
        

    ### Mileage ###
    try:
        mileage = int(quick_fact_elements[2].text[:-3].replace(',', ''))
        if mileage == "-":
            mileage = None
    except IndexError:
        mileage = None
        
    vehicle_usage_section = driver.find_elements(By.CSS_SELECTOR, "section[data-testid='VehicleUsageSection'] span")

    ### Vehicle usage section ###
    try:
        vehicle_usage_section = driver.find_elements(By.CSS_SELECTOR, "section[data-testid='VehicleUsageSection'] span")
    except NoSuchElementException:
        print("Vehicle usage elements not found")
    try:
        mileage_analysis = vehicle_usage_section[1].text[:-2].lower()
    except IndexError:
        mileage_analysis = None
        
    ### Transmission ###
    try:
        transmission_type = quick_fact_elements[4].text.lower()
        if transmission_type == "-":
            transmission_type = None
    except IndexError:
        transmission_type = None



    ### Drivetrain ###
    try:
        drivetrain = quick_fact_elements[8].text.lower()
        if drivetrain == "-":
            drivetrain = None
    except IndexError:
        drivetrain = None

    ### Fuel type ###
    try:
        fuel_type = quick_fact_elements[10].text.lower()
        if fuel_type == "-":
            fuel_type = None
    except IndexError:
        fuel_type = None
        
    try:
        vehicle_details_section_h3 = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='vehicleDetails'] h3")
    except NoSuchElementException:
        print("Vehicle details section elements not found")
        

    # Initialize variables to None
    make = model = year_of_manufacture = trim = variant = body_type = colour = seats = doors = None
    features_list = []
    extra_list = []
    # List of special characters
    special_characters = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    # Predefined list of car features
    feature_list = ["air conditioning", "cruise control", "parking assistant", "navigation system", "bluetooth", "alloy wheels", "trailer hitch", "push button start", "sunroof"]


    for i in range(len(vehicle_details_section_h3)):
        if vehicle_details_section_h3[i].text == "Overview":
            ### Vehicle details overview ###
            try:
                vehicle_details_elements_overview = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='section{}'] span".format(i))
                vehicle_details_elements_overview_length = len(vehicle_details_elements_overview)
            except NoSuchElementException:
                print("Vehicle details overview elements not found")
                

            for i in range(vehicle_details_elements_overview_length):
                if vehicle_details_elements_overview[i].text == 'Make: ':
                    make = vehicle_details_elements_overview[i+1].text.lower()
                    if make == "-":
                        make = None
                if vehicle_details_elements_overview[i].text == 'Model: ':
                    model = vehicle_details_elements_overview[i+1].text.lower()
                    if model == "-":
                        model = None
                if vehicle_details_elements_overview[i].text == 'Year: ':
                    year_of_manufacture = vehicle_details_elements_overview[i+1].text.lower()
                    if year_of_manufacture == "-":
                        year_of_manufacture = None
                    year_of_manufacture = int(year_of_manufacture)
                if vehicle_details_elements_overview[i].text == 'Trim: ':
                    trim = vehicle_details_elements_overview[i+1].text.lower()
                    if trim == "-":
                        trim = None
                    elif any(char in special_characters for char in trim) and trim != "-":
                        trim = find_string_before_special(trim)
                if vehicle_details_elements_overview[i].text == 'Variant: ':
                    variant = vehicle_details_elements_overview[i+1].text.lower()
                    if variant == "-":
                        variant = None
                    elif any(char in special_characters for char in variant) and variant != "-":
                        variant = find_string_before_special(variant)
                if vehicle_details_elements_overview[i].text == 'Body type: ':
                    body_type = vehicle_details_elements_overview[i+1].text.lower()
                    if body_type == "-":
                        body_type = None
                if vehicle_details_elements_overview[i].text == 'Colour: ':
                    colour = vehicle_details_elements_overview[i+1].text.lower()
                    if colour == "-":
                        colour = None


        elif vehicle_details_section_h3[i].text == "Dimensions & weights":
            #click on the h3 Dimensions & weights section to expand the section
            vehicle_details_section_h3[i].click()
            try:
                vehicle_details_dim_weights = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='section{}'] span".format(i))
                vehicle_details_dim_weights_length = len(vehicle_details_dim_weights)
            except NoSuchElementException:
                print("Vehicle details elements not found")

            for i in range(vehicle_details_dim_weights_length):
                if vehicle_details_dim_weights[i].text == 'Seats: ':
                    seats = vehicle_details_dim_weights[i+1].text
                    if seats == "-":
                        seats = None
                    seats = int(seats)
                if vehicle_details_dim_weights[i].text == 'Door count: ':
                    doors = vehicle_details_dim_weights[i+1].text[2:3]
                    if doors == "-":
                        doors = None
                    doors = int(doors)
            
        elif vehicle_details_section_h3[i].text == "Features":
            features_to_consider = ["air conditioning", "cruise control", "parking assistant", "navigation system", "bluetooth", "alloy wheels", "trailer hitch","push button start", "sunroof"]
            ### Vehicle features ###
            try:
                vehicle_feat_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='section{}'] span".format(i))
            except NoSuchElementException:
                print("Vehicle feature elements not found")
            # check if vehicle is not empty 
            if len(vehicle_feat_elements) > 0:
                # append the features to a list
                for span in vehicle_feat_elements:
                    inner_html_feat = span.get_attribute('innerHTML')
                    features_list.append(inner_html_feat.lower())
                    
                    
        elif vehicle_details_section_h3[i].text == "Extras":
            ### Vehicle extra ###
            try:
                vehicle_extra_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='section{}'] span".format(i))
            except NoSuchElementException:
                print("Vehicle extra elements not found")
            # check if vehicle extra is not empty 
            if len(vehicle_extra_elements) > 0:
                # append all the extra to a column
                for span in vehicle_extra_elements:
                    inner_html_extra = span.get_attribute('innerHTML')
                    if inner_html_extra[:3].lower() == 'a/c':
                        extra_list.append("air conditioning")
                    else:
                        extra_list.append(inner_html_extra.lower())



    all_features_the_car_has = features_list + extra_list
    all_features_the_car_has_set = set(all_features_the_car_has)
    features_to_consider_set = set(feature_list)
    features_we_care = list(all_features_the_car_has_set.intersection(features_to_consider_set))


    secondary_features = check_car_features(features_we_care,features_to_consider)

    primary_features = {
        'Make': make,
        'Model': model,
        'Trim': trim,
        'Variant': variant,
        'Body_type': body_type,
        'Transmission': transmission_type,
        'Drivetrain': drivetrain,
        'Fuel_type': fuel_type,
        'Year_of_manufacture': year_of_manufacture,
        'Mileage_in_KM': mileage,
        'Mileage_analysis': mileage_analysis,
        'Condition': condition,
        'Number_of_seats': seats,
        'Number_of_doors': doors,
        'Colour': colour,
        'Province': province,
        'Price_analysis': price_analysis,
    }

    car_data_dict = {**primary_features, **secondary_features}

    # adding the price last
    car_data_dict['Price'] = price
    
            
    # Query to find an existing car in the database
    existing_car = session.query(Car).filter(
        Car.Make == car_data_dict.get('Make', None),
        Car.Model == car_data_dict.get('Model', None),
        Car.Trim == car_data_dict.get('Trim', None),
        Car.Variant == car_data_dict.get('Variant', None),
        Car.Body_type == car_data_dict.get('Body_type', None),
        Car.Transmission == car_data_dict.get('Transmission', None),
        Car.Drivetrain == car_data_dict.get('Drivetrain', None),
        Car.Fuel_type == car_data_dict.get('Fuel_type', None),
        Car.Year_of_manufacture == car_data_dict.get('Year_of_manufacture', None),
        Car.Mileage_in_KM == car_data_dict.get('Mileage_in_KM', None),
        Car.Mileage_analysis == car_data_dict.get('Mileage_analysis', None),
        Car.Condition == car_data_dict.get('Condition', None),
        Car.Number_of_seats == car_data_dict.get('Number_of_seats', None),
        Car.Number_of_doors == car_data_dict.get('Number_of_doors', None),
        Car.Colour == car_data_dict.get('Colour', None),
        Car.Province == car_data_dict.get('Province', None),
        Car.Price_analysis == car_data_dict.get('Price_analysis', None),
        Car.Has_air_conditioning == car_data_dict.get('Has_air_conditioning', False),
        Car.Has_cruise_control == car_data_dict.get('Has_cruise_control', False),
        Car.Has_parking_assistant == car_data_dict.get('Has_parking_assistant', False),
        Car.Has_navigation_system == car_data_dict.get('Has_navigation_system', False),
        Car.Has_bluetooth == car_data_dict.get('Has_bluetooth', False),
        Car.Has_alloy_wheels == car_data_dict.get('Has_alloy_wheels', False),
        Car.Has_trailer_hitch == car_data_dict.get('Has_trailer_hitch', False),
        Car.Has_push_button_start == car_data_dict.get('Has_push_button_start', False),
        Car.Has_sunroof == car_data_dict.get('Has_sunroof', False),
        Car.Price == car_data_dict.get('Price', None),
    ).first()
    
    # If the car doesn't exist, add it
    if existing_car is None:
        new_car = Car(
            Make = car_data_dict.get('Make', None),
            Model = car_data_dict.get('Model', None),
            Trim = car_data_dict.get('Trim', None),
            Variant = car_data_dict.get('Variant', None),
            Body_type = car_data_dict.get('Body_type', None),
            Transmission = car_data_dict.get('Transmission', None),
            Drivetrain = car_data_dict.get('Drivetrain', None),
            Fuel_type = car_data_dict.get('Fuel_type', None),
            Year_of_manufacture = car_data_dict.get('Year_of_manufacture', None),
            Mileage_in_KM = car_data_dict.get('Mileage_in_KM', None),
            Mileage_analysis = car_data_dict.get('Mileage_analysis', None),
            Condition = car_data_dict.get('Condition', None),
            Number_of_seats = car_data_dict.get('Number_of_seats', None),
            Number_of_doors = car_data_dict.get('Number_of_doors', None),
            Colour = car_data_dict.get('Colour', None),
            Province = car_data_dict.get('Province', None),
            Price_analysis = car_data_dict.get('Price_analysis', None),
            Has_air_conditioning = car_data_dict.get('Has_air_conditioning', False),
            Has_cruise_control = car_data_dict.get('Has_cruise_control', False),
            Has_parking_assistant = car_data_dict.get('Has_parking_assistant', False),
            Has_navigation_system = car_data_dict.get('Has_navigation_system', False),
            Has_bluetooth = car_data_dict.get('Has_bluetooth', False),
            Has_alloy_wheels = car_data_dict.get('Has_alloy_wheels', False),
            Has_trailer_hitch = car_data_dict.get('Has_trailer_hitch', False),
            Has_push_button_start = car_data_dict.get('Has_push_button_start', False),
            Has_sunroof = car_data_dict.get('Has_sunroof', False),
            Price = car_data_dict.get('Price', None),
        )
        session.add(new_car)
        session.commit()
    else:
        print("This car already exists in the database. Skipping.")

    # TODO: Fix the issue that ends in TimeoutException and StaleElementReferenceException 
    # Go back to the search results
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button/span[text()='Back to search results']"))).click()

    
    # Increment the article index
    article_index += 1

    # Scroll down to load more articles
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@data-testid="SearchResultListItem"]')))
    # Refresh the main_div to get updated articles
    main_div = driver.find_element(By.XPATH, '//*[@data-testid="ListItemPage-0"]')




In [ ]:
session.close()